In [1]:
# Imports. To be cleaned as well

import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
from shapely.geometry import box
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.coords import BoundingBox
from rasterio.features import geometry_mask
import json
import numpy as np
import glob
import os
from rasterio.windows import from_bounds
import pathlib
path = r'C:\Users\Kostas\Downloads\S2A_MSIL2A_20170420T103021_N0204_R108_T32UNB_20170420T103454.SAFE\GRANULE\L2A_T32UNB_A009543_20170420T103454\IMG_DATA\R10m\L2A_T32UNB_20170420T103021_B02_10m.jp2'

In [5]:
from zipfile import ZipFile
import fnmatch
from rasterio.merge import merge

# Create a list of the existing Sentinel-2 zip files
sentinel_2_directory = pathlib.Path(r'C:\Users\Kostas\Desktop\GIMA\Module_7\Data\Sentinel2_images')
sentinel_2_zip_list = glob.glob(str(sentinel_2_directory) + '/*.zip', recursive=True)

s2zip = r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\Sentinel2_images\1.S2A_MSIL2A_20200921T103031_N0214_R108_T32UNB_20200921T151006.zip"
# Create a list of the band paths of Sentinel-2 zips
#for s2zip in sentinel_2_zip_list:
zipaki = s2zip
print('---------------')

# This part is a filter to find only the relevant 60m bands from inside the Sentinel-2 zip. Maybe I should wrap this up into a function as well.
with ZipFile(zipaki, 'r') as zipObj:
    file_list = zipObj.namelist()
    pattern = '*/R60m/*B???60m.jp2'

    filtered_list = []
    for file in file_list:
        if fnmatch.fnmatch(file, pattern):
            filtered_list.append(file)
            print(file)
            # Create a list sorted by ascending band no
            band_no = file[-10] + file[-9]
            print(band_no)
            file = pathlib.Path("zip+file:" + zipaki + '!/' + file)
            if band_no == '01':
                band1 = file
            elif band_no == '02':
                band2 = file
            elif band_no == '03':
                band3 = file
            elif band_no == '04':
                band4 = file
            elif band_no == '05':
                band5 = file
            elif band_no == '06':
                band6 = file
            elif band_no == '07':
                band7 = file
            elif band_no == '8A':
                band8 = file
            elif band_no == '09':
                band9 = file
            elif band_no == '11':
                band11 = file
            elif band_no == '12':
                band12 = file
            band_paths = [band1, band2, band3, band4, band5, band6, band7, band8, band9, band11, band12]
    print(f'List of spectral bands for file {zipaki} completed.')


bands = []
for band_path in band_paths:
        bands.append(rasterio.open(band_path, driver='JP2OpenJPEG'))


# Merge all the band rasters into a single multiband raster
#merged, merged_transform = merge(bands)

output_path = pathlib.Path(r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\Sentinel2_images\test\test.tiff")

all = rasterio.open(output_path, 'w', driver='GTiff',
                          width=bands[1].width, height=bands[1].height,
                          count=11,
                          crs=bands[1].crs,
                          transform=bands[1].transform,
                          dtype='uint16')

for i in range(0, len(bands)):
    all.write(bands[i].read(1), i+1)

# Close the bands that were opened with rasterio
# Close multi band image
all.close()
# Close single bands
for band in bands:
    band.close()



# rgb_path = pathlib.Path(r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\Sentinel2_images\test\rgb.tiff")

# # Test with an rgb image
# rgb = rasterio.open(rgb_path, 'w', driver='PNG',
#                           width=bands[1].width, height=bands[1].height,
#                           count=3,
#                           crs=bands[1].crs,
#                           transform=bands[1].transform,
#                           dtype='uint16')
# rgb.write(bands[3].read(1),1)
# rgb.write(bands[2].read(1),2)
# rgb.write(bands[1].read(1),3)
# rgb.close()

# merged_meta = bands[0].meta.copy()
# merged_meta.update(count=len(bands), transform=merged_transform)
# # Write the merged raster to the output file
# with rasterio.open(output_path, "w", **merged_meta) as dest:
#     dest.write(merged)

# # Close all the band rasters
# for band in bands:
#     band.close()       

# ------- THIS IS UNTOUCHED YET -------------
#     # Now read each band (each band is a different raster)
#     for raster in filtered_list:
#         zipped_image = pathlib.Path("zip+file:" + zipaki + '!/' + raster)
#         # Spatial filter. Has all the envelope coordinates that are within a Sentinel-2 image
#         minx_list, miny_list, maxx_list, maxy_list = getContainedEnvelopeCoords(zipped_image, envelope_list)
#         raster_name = imageNaming(raster)
#         print("\tCreating images around PEP725 stations for the band:", raster_name)

#         # For each envelope in the band, export the image and use the clc related info to mask the pixels
#         for j in range(0, len(minx_list)):
#             # Temporal filter should probably go here
#             output_file_name = os.path.join(output_path, raster_name + f'_Patch_{j+1}.png')
#             exportImage(zipped_image, output_file_name, minx_list[j], miny_list[j], maxx_list[j], maxy_list[j])
#             pixelMasking(output_file_name, clc_path, clc_pixel_values, output_file_name)
#         print(f'\tPatch pixel masking completed for all patches of the band {raster_name}\n')
#     print(f'\tImage extraction completed for the file{zipaki}')
# print('Image extraction completed for all the files')
    


---------------
S2A_MSIL2A_20200921T103031_N0214_R108_T32UNB_20200921T151006.SAFE/GRANULE/L2A_T32UNB_A027418_20200921T103107/IMG_DATA/R60m/T32UNB_20200921T103031_B01_60m.jp2
01


NameError: name 'band2' is not defined

In [ ]:
'''
This function is used to receive a string from the raster file's name
using the split() method. It splits the string wherever an underscore appears and then accesses the second-to-last element.
'''
def imageNaming(raster_path):
    string_parts = raster_path.split("_")
    band_string = string_parts[-3]+ "_" + string_parts[-2] + "_" + string_parts[-1]
    band_string = band_string.replace('.jp2','')
    return band_string

# Example output: 20170420T103021_B02_10m